0. Environment configuration

In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install SQLAlchemy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
from sqlalchemy import  create_engine

In [4]:
df_dates = pd.read_csv('date_info.csv')

df_dates['calendar_date'] = pd.to_datetime(df_dates['calendar_date'],dayfirst=True)

df_rest_visitor = pd.read_csv('restaurants_visitors.csv')

df_rest_visitor = df_rest_visitor.loc[df_rest_visitor['visit_date'] != "#VALUE!"]

df_rest_visitor['visit_date'] = pd.to_datetime(df_rest_visitor['visit_date'],dayfirst=True)

df_store_info = pd.read_csv('store_info.csv')

In [5]:
print (df_rest_visitor)

                     id visit_date    visit_datetime  reserve_datetime  \
0      db80363d35f10926 2016-01-01  01/01/2016 00:00  01/01/2016 01:00   
1      db80363d35f10926 2016-01-02  02/01/2016 01:00  01/01/2016 16:00   
2      db80363d35f10926 2016-01-02  02/01/2016 01:00  01/01/2016 15:00   
3      db80363d35f10926 2016-01-02  02/01/2016 18:00  02/01/2016 14:00   
4      db80363d35f10926 2016-01-02  02/01/2016 18:00  02/01/2016 02:00   
...                 ...        ...               ...               ...   
12157  3cad29d1a23209d2 2017-05-27  27/05/2017 21:00  09/02/2017 18:00   
12158  1033310359ceeac1 2017-05-28  28/05/2017 23:00  15/04/2017 22:00   
12159  1033310359ceeac1 2017-05-28  28/05/2017 23:00  18/04/2017 17:00   
12160  1033310359ceeac1 2017-05-30  30/05/2017 23:00  19/04/2017 17:00   
12161  3cad29d1a23209d2 2017-05-31  31/05/2017 21:00  22/04/2017 10:00   

       reserve_visitors  
0                     5  
1                     2  
2                     4  
3      

In [6]:
print(df_dates)

    calendar_date day_of_week  holiday_flg
0      2016-01-01      Friday            1
1      2016-01-02    Saturday            1
2      2016-01-03      Sunday            1
3      2016-01-04      Monday            0
4      2016-01-05     Tuesday            0
..            ...         ...          ...
512    2017-05-27    Saturday            0
513    2017-05-28      Sunday            0
514    2017-05-29      Monday            0
515    2017-05-30     Tuesday            0
516    2017-05-31   Wednesday            0

[517 rows x 3 columns]


In [7]:
print(df_store_info)

             store_id             genre_name  \
0    0f0cdeee6c9bf3d7         Italian/French   
1    7cc17a324ae5c7dc         Italian/French   
2    fee8dcf4d619598e         Italian/French   
3    a17f0778617c76e2         Italian/French   
4    83db5aff8f50478e         Italian/French   
..                ...                    ...   
824  9bf595ef095572fb  International cuisine   
825  764f71040a413d4d                  Asian   
826  10bbe8acd943d8f6                  Asian   
827  7514d90009613cd6          Karaoke/Party   
828  c8fe396d6c46275d          Karaoke/Party   

                                 area_name   latitude   longitude  
0             Hyōgo-ken Kōbe-shi Kumoidōri  34.695124  135.197853  
1             Hyōgo-ken Kōbe-shi Kumoidōri  34.695124  135.197853  
2             Hyōgo-ken Kōbe-shi Kumoidōri  34.695124  135.197853  
3             Hyōgo-ken Kōbe-shi Kumoidōri  34.695124  135.197853  
4             Tōkyō-to Minato-ku Shibakōen  35.658068  139.751599  
..             

In [8]:
db_url = "sqlite:///:memory:"
engine = engine = create_engine(db_url)

df_dates.to_sql('dates', engine, index=False)
df_rest_visitor.to_sql('restaurant_visitors', engine, index=False)
df_store_info.to_sql('store_info',engine, index=False)

829

1. Write the SQL queries necessary to generate a list of the five restaurants that have the highest average number of visitors on holidays. The result table should also contain that average per restaurant

In [9]:
query_1 = """
SELECT 
    a.[id] as restaurant,
    AVG(a.[reserve_visitors]) as average_of_visitors 
FROM restaurant_visitors as a 
LEFT JOIN dates AS b 
ON a. [visit_date] = b. [calendar_date] 
WHERE b.[holiday_flg]=1 
GROUP BY a.[id] 
ORDER BY average_of_visitors DESC 
LIMIT 5
"""

first_result = pd.read_sql_query(query_1,engine)

print (first_result)

         restaurant  average_of_visitors
0  db80363d35f10926             7.275000
1  bb09595bab7d5cfb             5.833333
2  e053c561f32acc28             5.240000
3  24b9b2a020826ede             4.333333
4  42c9aa6d617c5057             4.228571


2. Use SQL to discover which day of the week there are usually more visitors on average in restaurants.



In [10]:
query_2 = """
SELECT 
    b.[day_of_week] ,
    AVG(a.[reserve_visitors]) as average_of_visitor 
FROM dates AS b 
LEFT JOIN restaurant_visitors AS a
ON b. [calendar_date] = a. [visit_date] 
GROUP BY b.[day_of_week] 
ORDER BY average_of_visitor DESC
"""

second_result = pd.read_sql_query(query_2,engine)

print (second_result)

# The answer is Friday

  day_of_week  average_of_visitor
0      Friday            4.454754
1   Wednesday            4.216495
2    Thursday            4.115640
3      Monday            4.049140
4    Saturday            3.983149
5     Tuesday            3.913649
6      Sunday            3.492447


3. How was the percentage of growth of the amount of visitors week over week for the last four weeks of the data? Use SQL too

In [11]:
query_days = """
WITH amount_wow AS ( 
    SELECT
        DATE(b.calendar_date, '-' || strftime('%w', b.calendar_date) || ' days') AS week_start, 
        SUM(a.reserve_visitors) AS amount_visitors 
    FROM dates AS b
    LEFT JOIN restaurant_visitors AS a 
    ON b.calendar_date = a.visit_date
    GROUP BY week_start) 
SELECT  
    week_start,
    amount_visitors,
    LAG(amount_visitors) OVER (ORDER BY week_start) AS previous_week_amount,
    (amount_visitors * 1.00 -LAG(amount_visitors) OVER (ORDER BY week_start))/LAG(amount_visitors) OVER (ORDER BY week_start)  AS  growth_visitors_wow
FROM amount_wow 
ORDER BY week_start DESC 
LIMIT 4
"""

days_result = pd.read_sql_query(query_days,engine)

print (days_result)

   week_start  amount_visitors  previous_week_amount  growth_visitors_wow
0  2017-05-28               15                    63            -0.761905
1  2017-05-21               63                    78            -0.192308
2  2017-05-14               78                   175            -0.554286
3  2017-05-07              175                   164             0.067073
